In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/home/jvdzwaan/data/icdar2017st/eng_periodical/wordmapping_gs_vs_ocr-lowercase.csv', index_col=0, encoding='utf-8')
df = df.fillna(u'@@@')
print df.shape

In [ ]:
# Number of errors
df[df['gs'] != df['ocr']].shape

In [ ]:
import codecs

with codecs.open('/home/jvdzwaan/code/english-words/words_alpha.txt', encoding='utf-8') as f:
    words = [w.strip() for w in f.readlines()]
print 'number of words', len(words)

In [ ]:
%%time
words = set(words)

In [ ]:
errors = df[df['gs'] != df['ocr']]

In [ ]:
%%time
errors.apply(lambda row: row['gs'] in words, axis=1)

In [ ]:
%%time
from ochre.ocrerrors import categorize_errors

errors = df[df['gs'] != df['ocr']]
#print errors
result = categorize_errors(errors, terms=words)

In [ ]:
from ochre.rmgarbage import get_rmgarbage_errors

def garbage_string_error(row, gs_name='gs', ocr_name='ocr', empty_word='@@@'):
    #ocr = 'CslwWkrm bla'
    #gs = 'bla'
    gs = row[gs_name]
    ocr = row[ocr_name]
    
    if ocr != empty_word:
        words = ocr.split()
        for w in words:
            errors = get_rmgarbage_errors(w)
            if len(errors) > 0:
                return True
    return False
    
errors['garbage'] = errors.apply(garbage_string_error, axis=1)

In [ ]:
get_rmgarbage_errors('telog\'lpn')

In [ ]:
print words[4536]

In [ ]:
repr(words[4536])

In [ ]:
print 'adiaphorite' in words

In [ ]:
result

In [ ]:
result[['ocr', 'error_type']].groupby('error_type').count().plot.barh(figsize=(15,7))

In [ ]:
result[['ocr', 'error_type']].groupby('error_type').count()

In [ ]:
result.query('error_type == "whitespace"')

In [ ]:
from ochre.ocrerrors import get_error_types

def find_multiple(row, error_types):
    num_true = 0
    for err_name, err_function in error_types.items():
        if row[err_name] == True:
            num_true += 1
    if num_true > 1:
        print row
        return 1
    return 0
duplicate_errors = result.apply(find_multiple, error_types=get_error_types(), axis=1)    

In [ ]:
sum(duplicate_errors)

In [ ]:
result